## Training Corpus
We first create our corpus (aka Training Data)

In [1]:
corpus = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?",
]

for doc in corpus:
    print(doc)

This is the first document.
This document is the second document.
And this is the third one.
Is this the first document?


## Initial Vocabulary
Now we must create the initial vocabulary which will have our unique characters

create a list of unique characters

In [2]:
# (no duplicates are allowed in sets)
unique_chars = set()

# add chars from corpus to set
for doc in corpus:
    for char in doc:
        unique_chars.add(char)

In [3]:
print(unique_chars)

{'.', 'o', 'r', 'T', 's', 'n', 'i', 'u', 'f', 'e', 'h', 'A', 'c', ' ', '?', 't', 'd', 'I', 'm'}


we now convert it into a list.
(sets are immutable and cannot be indexed)

In [4]:
vocab = list(unique_chars)
vocab.sort()        # simply, for coninstency and repoducibility

In [5]:
print(vocab)

[' ', '.', '?', 'A', 'I', 'T', 'c', 'd', 'e', 'f', 'h', 'i', 'm', 'n', 'o', 'r', 's', 't', 'u']


add an **end of word** token.  
> so the model will be able to differentiate between words and avoid irrelevant/wrong pairs of characters.

In [6]:
end_of_word = '/<w>'
vocab.append(end_of_word)

In [7]:
print('Initial Vocabulary:')
print(vocab)
print(f'size: {len(vocab)}')

Initial Vocabulary:
[' ', '.', '?', 'A', 'I', 'T', 'c', 'd', 'e', 'f', 'h', 'i', 'm', 'n', 'o', 'r', 's', 't', 'u', '/<w>']
size: 20


## Pre-Tokenization

here, we will split the corpus into words, then characters.
- to split into words, we'll use the space character
- we will add `</w>>` at the end of each word

In [8]:
word_splits = {}

for doc in corpus:

    # splitting by ' ' character
    words = doc.split(' ')

    for word in words:
        
        char_list = list(word) + [end_of_word]      # convert words into list and append the char

        # convert to list because we will need an immutable object to act as a key in the dictionary
        word_tuple = tuple(char_list)

        if word_tuple not in word_splits:
            word_splits[word_tuple] = 0
        word_splits[word_tuple] += 1                # incrememnting count for each word when found

print('\nThe final dictionary with word count:')
print(word_splits)


The final dictionary with word count:
{('T', 'h', 'i', 's', '/<w>'): 2, ('i', 's', '/<w>'): 3, ('t', 'h', 'e', '/<w>'): 4, ('f', 'i', 'r', 's', 't', '/<w>'): 2, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '.', '/<w>'): 2, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '/<w>'): 1, ('s', 'e', 'c', 'o', 'n', 'd', '/<w>'): 1, ('A', 'n', 'd', '/<w>'): 1, ('t', 'h', 'i', 's', '/<w>'): 2, ('t', 'h', 'i', 'r', 'd', '/<w>'): 1, ('o', 'n', 'e', '.', '/<w>'): 1, ('I', 's', '/<w>'): 1, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '?', '/<w>'): 1}


## Helper Functions: 

### get_pair_stats()
This function will pair the adjecent characters and count their frequency.  
example:  
**input** =  
``` {('T', 'h', 'i', 's', '</w>'): 2, ('i', 's', '</w>'): 2, ...} ```
  
**output** =  
``` # {('i', 's'): 4, ('s', '</w>'): 4, ('T', 'h'): 2, ...} ```


In [9]:
import collections

def get_pair_stats(splits):
    # A collection's dictionary will create a new key if it already doesn't exist in the dictionary.
    pair_counts = collections.defaultdict(int)      #defaultdict will have default values of 0

    for word_tuple, freq in splits.items():
        symbols = list(word_tuple)                  # converting tuple to list

        for i in range(len(symbols)-1):               # iterating through each element in the word
            pair = (symbols[i], symbols[i+1])       # pairing chars with the next char
            pair_counts[pair] += freq               # addin the frequency of the word
    
    return pair_counts


In [10]:
pair_counts_dict = get_pair_stats(word_splits)
print(pair_counts_dict)

defaultdict(<class 'int'>, {('T', 'h'): 2, ('h', 'i'): 5, ('i', 's'): 7, ('s', '/<w>'): 8, ('t', 'h'): 7, ('h', 'e'): 4, ('e', '/<w>'): 4, ('f', 'i'): 2, ('i', 'r'): 3, ('r', 's'): 2, ('s', 't'): 2, ('t', '/<w>'): 3, ('d', 'o'): 4, ('o', 'c'): 4, ('c', 'u'): 4, ('u', 'm'): 4, ('m', 'e'): 4, ('e', 'n'): 4, ('n', 't'): 4, ('t', '.'): 2, ('.', '/<w>'): 3, ('s', 'e'): 1, ('e', 'c'): 1, ('c', 'o'): 1, ('o', 'n'): 2, ('n', 'd'): 2, ('d', '/<w>'): 3, ('A', 'n'): 1, ('r', 'd'): 1, ('n', 'e'): 1, ('e', '.'): 1, ('I', 's'): 1, ('t', '?'): 1, ('?', '/<w>'): 1})


Now we have the frequency of occurence of each pair of characters in the corpus.

### merge_pair()

* Now we will merge the pairs with the most frequencies into a new token.  
* Addionally, we will keep a track of these merges so we can undo everything when needed.

In [11]:
def merge_pair(pair_to_merge, splits):
    new_splits = {}
    (first_char, second_char) = pair_to_merge
    merged_token = first_char + second_char

    for word_tuple, freq in splits.items():
        symbols = list(word_tuple)
        new_symbols = []

        i=0
        while i < len(symbols):
            if i < len(symbols) - 1 and symbols[i] == first_char and symbols[i+1] == second_char:
                new_symbols.append(merged_token)
                i+=2        # to skip to the next symbol
            else:
                new_symbols.append(symbols[i])
                i+=1
        new_splits[tuple(new_symbols)] = freq
    return new_splits


In [12]:
merge_pair(('i','s'), pair_counts_dict)

{('T', 'h'): 2,
 ('h', 'i'): 5,
 ('is',): 7,
 ('s', '/<w>'): 8,
 ('t', 'h'): 7,
 ('h', 'e'): 4,
 ('e', '/<w>'): 4,
 ('f', 'i'): 2,
 ('i', 'r'): 3,
 ('r', 's'): 2,
 ('s', 't'): 2,
 ('t', '/<w>'): 3,
 ('d', 'o'): 4,
 ('o', 'c'): 4,
 ('c', 'u'): 4,
 ('u', 'm'): 4,
 ('m', 'e'): 4,
 ('e', 'n'): 4,
 ('n', 't'): 4,
 ('t', '.'): 2,
 ('.', '/<w>'): 3,
 ('s', 'e'): 1,
 ('e', 'c'): 1,
 ('c', 'o'): 1,
 ('o', 'n'): 2,
 ('n', 'd'): 2,
 ('d', '/<w>'): 3,
 ('A', 'n'): 1,
 ('r', 'd'): 1,
 ('n', 'e'): 1,
 ('e', '.'): 1,
 ('I', 's'): 1,
 ('t', '?'): 1,
 ('?', '/<w>'): 1}

The `merge_pair()` function has created a new "is" token and it appears 7 times in the corpus

# BPE Merging Loop

In [13]:
num_merges = 15
merges = {}
current_splits = word_splits.copy()

print('Starting BPE Merges')
print(f'Initial Splits{current_splits}')
print('-'*30)

Starting BPE Merges
Initial Splits{('T', 'h', 'i', 's', '/<w>'): 2, ('i', 's', '/<w>'): 3, ('t', 'h', 'e', '/<w>'): 4, ('f', 'i', 'r', 's', 't', '/<w>'): 2, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '.', '/<w>'): 2, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '/<w>'): 1, ('s', 'e', 'c', 'o', 'n', 'd', '/<w>'): 1, ('A', 'n', 'd', '/<w>'): 1, ('t', 'h', 'i', 's', '/<w>'): 2, ('t', 'h', 'i', 'r', 'd', '/<w>'): 1, ('o', 'n', 'e', '.', '/<w>'): 1, ('I', 's', '/<w>'): 1, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '?', '/<w>'): 1}
------------------------------


In [14]:
for i in range(num_merges):
    print(f'Merge iteration {i+1}/{num_merges}')

    # calculate pair frequencies
    pair_stats = get_pair_stats(current_splits)
    
    if not pair_stats:
        print('No more pairs to merge.')
        break

    # find best pair
    best_pair = max(pair_stats, key=pair_stats.get)     # so that it will check for max frequency instead of the key value
    best_freq = pair_stats[best_pair]
    print(f'Found Best Pair: {best_pair} with Frequency {best_freq}')

    # merge the best pair
    current_splits = merge_pair(best_pair, current_splits)
    new_token = best_pair[0] + best_pair[1]
    print(f'Merging {best_pair} into `{new_token}`')
    print(f'splits after merge: {current_splits}')

    # update vocab
    vocab.append(new_token)
    print(f'Updated Vocab: {vocab}')

    # store the merge
    merges[best_pair] = new_token
    print(f'Updated Merges: {merges}')

    print('-'*30)

Merge iteration 1/15
Found Best Pair: ('s', '/<w>') with Frequency 8
Merging ('s', '/<w>') into `s/<w>`
splits after merge: {('T', 'h', 'i', 's/<w>'): 2, ('i', 's/<w>'): 3, ('t', 'h', 'e', '/<w>'): 4, ('f', 'i', 'r', 's', 't', '/<w>'): 2, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '.', '/<w>'): 2, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '/<w>'): 1, ('s', 'e', 'c', 'o', 'n', 'd', '/<w>'): 1, ('A', 'n', 'd', '/<w>'): 1, ('t', 'h', 'i', 's/<w>'): 2, ('t', 'h', 'i', 'r', 'd', '/<w>'): 1, ('o', 'n', 'e', '.', '/<w>'): 1, ('I', 's/<w>'): 1, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '?', '/<w>'): 1}
Updated Vocab: [' ', '.', '?', 'A', 'I', 'T', 'c', 'd', 'e', 'f', 'h', 'i', 'm', 'n', 'o', 'r', 's', 't', 'u', '/<w>', 's/<w>']
Updated Merges: {('s', '/<w>'): 's/<w>'}
------------------------------
Merge iteration 2/15
Found Best Pair: ('i', 's/<w>') with Frequency 7
Merging ('i', 's/<w>') into `is/<w>`
splits after merge: {('T', 'h', 'is/<w>'): 2, ('is/<w>',): 3, ('t', 'h', 'e', '/<w>'): 4, ('f',

In [15]:
print("\n--- BPE Merges Complete ---")
print(f"Final Vocabulary Size: {len(vocab)}")
print("\nLearned Merges (Pair -> New Token):")
# Pretty print merges
for pair, token in merges.items():
    print(f"{pair} -> '{token}'")

print("\nFinal Word Splits after all merges:")
print(current_splits)

print("\nFinal Vocabulary (sorted):")
# Sort for consistent viewing
final_vocab_sorted = sorted(list(set(vocab))) # Use set to remove potential duplicates if any step introduced them
print(final_vocab_sorted)


--- BPE Merges Complete ---
Final Vocabulary Size: 35

Learned Merges (Pair -> New Token):
('s', '/<w>') -> 's/<w>'
('i', 's/<w>') -> 'is/<w>'
('t', 'h') -> 'th'
('th', 'e') -> 'the'
('the', '/<w>') -> 'the/<w>'
('d', 'o') -> 'do'
('do', 'c') -> 'doc'
('doc', 'u') -> 'docu'
('docu', 'm') -> 'docum'
('docum', 'e') -> 'docume'
('docume', 'n') -> 'documen'
('documen', 't') -> 'document'
('i', 'r') -> 'ir'
('.', '/<w>') -> './<w>'
('d', '/<w>') -> 'd/<w>'

Final Word Splits after all merges:
{('T', 'h', 'is/<w>'): 2, ('is/<w>',): 3, ('the/<w>',): 4, ('f', 'ir', 's', 't', '/<w>'): 2, ('document', './<w>'): 2, ('document', '/<w>'): 1, ('s', 'e', 'c', 'o', 'n', 'd/<w>'): 1, ('A', 'n', 'd/<w>'): 1, ('th', 'is/<w>'): 2, ('th', 'ir', 'd/<w>'): 1, ('o', 'n', 'e', './<w>'): 1, ('I', 's/<w>'): 1, ('document', '?', '/<w>'): 1}

Final Vocabulary (sorted):
[' ', '.', './<w>', '/<w>', '?', 'A', 'I', 'T', 'c', 'd', 'd/<w>', 'do', 'doc', 'docu', 'docum', 'docume', 'documen', 'document', 'e', 'f', 'h', '